{{title_s1_5}}

So far in this tutorial, we've demonstrated how to read Sentinel-1 RTC imagery from two sources and assemble analysis-ready data cubes with appropriate metadata. Now, we'll perform a comparison of the two datasets. 

::::{tab-set}
:::{tab-item} Dataset comparison

While the two datasets are very similar, there are a few key differences:  
1) They use different sources images.   
    - ASF Sentinel-1 RTC imagery is processed from Single Look Complex ([SLC](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/single-look-complex)) images while Planetary Computer Sentinel-1 RTC imagery is processed from Ground Range Detected ([GRD](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/ground-range-detected)) images. SLC images contain both amplitude and phase information for each pixel. They are in radar coordinates and have not yet been multi-looked. In contrast, GRD images has been detected, multi-looked and projected to ground range.

2) They use different digital elevation models (DEMs) for terrain correction.  
    - ASF uses the [GLO-30 Copernicus DEM](https://dataspace.copernicus.eu/explore-data/data-collections/copernicus-contributing-missions/collections-description/COP-DEM) while Planetary Computer uses a Planet DEM.  
3) The datasets have different pixel spacings. For Planetary Computer, the pixel spacing is 10m in both range and azimuth directions. ASF has the option to produce images with 30 m, 20 m, or 10 m pixel spacaing. The data used in this tutorial is 30 m. Note that there are tradeoffs in processing time and file size with pixel spacing, see more discussion [here](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#pixel-spacing_1).  
4) Each platform uses a different algorithm for RTC processing.
5) The ASF dataset comes with an associated layover shadow map for each scene while the Planetary Computer dataset does not.  

All of the above information and much more detail about the processing methods for both datasets are available in each dataset's documentation pages:
- [ASF Sentinel-1 RTC Product Guide](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#pixel-spacing_1)  
- [Microsoft Planetary Computer Sentinel-1 RTC dataset](https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc) 
::: 

:::{tab-item} Outline

(content.Section_A)=
**[A. Read and prepare data](#a-read-and-prepare-data)**  
- {{a1_s1_nb5}}  

(content.Section_B)=
**[B. Ensure direct comparison between datasets](#b-ensure-direct-comparison-between-datasets)**
- {{b1_s1_nb5}}
- {{b2_s1_nb5}}
- {{b3_s1_nb5}}

(content.Section_C)=
**[C. Combine objects](#c-combine-objects)**
- {{c1_s1_nb5}}
- {{c2_s1_nb5}}

(content.Section_D)=
**[D. Visualize comparisons](#d-visualize-comparisons)**
- {{d1_s1_nb5}}
- {{d2_s1_nb5}}

:::
:::{tab-item} Learning goals

## Xarray and python techniques
- Conditional selection based on non-dimensional coordinates using `xr.Dataset.where()`
- Subsetting datasets based on dimensional coordinates using `xr.DataArray.isin()`
- Adding dimensional and non-dimensional coordinates to `xr.Dataset` objects
- Xarray plotting methods
- Projecting xarray objects to different grids using `xr.interp_like()`

## High-level science goals
- Comparing and evaluating multiple datasets

:::
::::


In [ ]:
%xmode minimal
import dask.distributed
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import stackstac
import xarray as xr
import pystac
import planetary_computer

from dask.distributed import Client

import s1_tools

%matplotlib inline

{{break}}

## A. Read and prepare data

At the end of notebook 3, we wrote the analysis-ready ASF Sentinel-1 data cube that had been clipped to a smaller spatial area of interest to disk. We'll read that into memory now to use in this comparison.

We used Jupyter cell magic to persist the Planetary Computer data cube created in noteboook 4. Now we can read it into our notebook by adding `-r` to the store magic command used to persist it. Read more about `storemagic` [here](https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html).

In [2]:
%store -r da
pc_cube = da

In [3]:
pc_cube = pc_cube.compute()

In [4]:
timeseries_type = 'full'

asf_cube = xr.open_dataset(f'../data/{timeseries_type}_timeseries/intermediate_cubes/s1_asf_clipped_cube.zarr',
                           engine='zarr',chunks='auto', decode_coords='all')

In [5]:
asf_cube = asf_cube.compute()

### {{a1_s1_nb5}}

First, make sure that both objects are projected to the same CRS.

In [6]:
assert pc_cube.rio.crs == asf_cube.rio.crs, 'CRS of both data cubes are expected to be identical.'

Let's also check how mising data is handled in both objects. We want missing data to be assigned NaN values.

In [ ]:
asf_cube['vv'].rio.nodata

In [8]:
pc_cube.sel(band='vv').rio.nodata

The `pc_cube` array contains nan values, but it doesn't have an encoding specifying what value is used to represent nodata. We can assign a nodata value to the dataset below. See Rioxarray's [Nodata Management documentation](https://corteva.github.io/rioxarray/stable/getting_started/nodata_management.html) for more detail on this.

In [ ]:
pc_cube.rio.write_nodata(np.nan, inplace=True)
pc_cube.rio.nodata

In [10]:
assert np.isnan(asf_cube.vh.rio.nodata) == np.isnan(pc_cube.sel(band='vh').rio.nodata) == True, 'Expected vh nodata value to be np.nan'
assert np.isnan(asf_cube.vv.rio.nodata) == np.isnan(pc_cube.sel(band='vv').rio.nodata) == True, 'Expected vv nodata value to be np.nan'

## B. Ensure direct comparison between datasets

Both objects have temporal dimensions with the same length, so we can be reasonably sure that these two objects have common acquisitions but we can do a few more things to be sure about that. 

Thanks to all of the metadata wrangling we did in earlier notebooks, we can quickly access variables like acquisition date and time, data take IDs and absolute orbit numbers to verify that the we're working with comparable objects.

### {{b1_s1_nb5}}

If we try to compare orbit numbers right away (eg: `asf_cube['abs_orbit_num'].data == pc_cube['sat:absolute_orbit']`), we will get that they are false. This is because the data in the underlying numpy arrays is currently not of the same type:

In [ ]:
print(asf_cube['abs_orbit_num'].data.dtype)
print(pc_cube['sat:absolute_orbit'].data.dtype)

Convert the absolute orbit array of `asf_cube` to an array of integers:

In [12]:
asf_cube['abs_orbit_num'] = asf_cube['abs_orbit_num'].astype(int)
                                                             

Now compare them:

In [ ]:
asf_cube['abs_orbit_num'].data == pc_cube['sat:absolute_orbit'].data

### {{b2_s1_nb5}}

The PC dataset has a higher spatial resolution than the ASF dataset, which you can see in the sizes of the x and y dimensions of the dataset. The ASF dataset masks out more pixels due to shadow than the PC dataset. To explore whether or not this is the cause of the observed backscatter offset between the two, we need to mask out the pixels in the PC dataset that are masked in the ASF dataset. 

In order to do this, first we must downsample the PC dataset to match the spatial resolution of the ASF dataset. There are multiple xarray methods that could be used but here we will use the `xr.interp_like()` method because we would like the PC x and y dimensions to match the ASF dimension exactly. 

In [14]:
asf_cube = asf_cube.rename({"acq_date": "time"})

In [15]:
asf_da = asf_cube.to_dataarray(dim='band')

In [16]:
pc_downsample = pc_cube.interp_like(asf_da)

### {{b3_s1_nb5}}

In [17]:
pc_mask = xr.where(asf_da.notnull(), pc_downsample, np.nan)

## C. Combine objects

### {{c1_s1_nb5}}

In [18]:
asf_da = asf_da.expand_dims('source')
pc_mask = pc_mask.expand_dims('source')

In [19]:
asf_da['source'] = ('source', ['asf'])
pc_mask['source'] = ('source', ['pc'])

### {{c2_s1_nb5}}

In [20]:
comparison_obj = xr.combine_by_coords([asf_da, pc_mask])

In [ ]:
comparison_obj

## D. Visualize comparisons

### {{d1_s1_nb5}}

Look at VV backscatter first:

In [ ]:
vv_fg = s1_tools.power_to_db(comparison_obj.sel(band='vv').isel(time=1)).plot(col='source', 
                                                cmap=plt.cm.Greys_r, cbar_kwargs=({'label':'dB'}))
vv_fg.fig.suptitle('Comparing VV backscatter from ASF and PC datasets')
vv_fg.fig.supxlabel('X coordinate of projection (m)')
vv_fg.fig.supylabel('Y coordinate of projection (m)')
vv_fg.fig.set_figheight(7)
vv_fg.fig.set_figwidth(12)

for i in range(len(vv_fg.axs[0])):
    
    vv_fg.axs[0][i].set_xlabel(None)
    vv_fg.axs[0][i].set_ylabel(None)
vv_fg.axs[0][0].set_title('ASF')
vv_fg.axs[0][1].set_title('PC')
;

Then VH:

In [ ]:
vh_fg = s1_tools.power_to_db(comparison_obj.sel(band='vh').isel(time=1)).plot(col='source', 
                                                cmap=plt.cm.Greys_r, cbar_kwargs=({'label':'dB'}))
vh_fg.fig.suptitle('Comparing VH backscatter from ASF and PC datasets')
vh_fg.fig.supxlabel('X coordinate of projection (m)')
vh_fg.fig.supylabel('Y coordinate of projection (m)')
vh_fg.fig.set_figheight(7)
vh_fg.fig.set_figwidth(12)
for i in range(len(vh_fg.axs[0])):
    
    vh_fg.axs[0][i].set_xlabel(None)
    vh_fg.axs[0][i].set_ylabel(None)
vh_fg.axs[0][0].set_title('ASF')
vh_fg.axs[0][1].set_title('PC')
;

### {{d2_s1_nb5}}

Instead of computing mean backscatter values along the time dimension, reduce along the spatial dimensions (x and y) to see backscatter variability over time:

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(14,8), layout='constrained')
s1_tools.power_to_db(comparison_obj.sel(source='asf',band='vv').mean(dim=['x','y'])).plot.scatter(x='time', ax=ax[0], label='asf',
                                                                                                  c='b', alpha=0.75)
s1_tools.power_to_db(comparison_obj.sel(source='pc',band='vv').mean(dim=['x','y'])).plot.scatter(x='time', ax=ax[0], label='pc',
                                                                                                 c='r',alpha=0.75)

s1_tools.power_to_db(comparison_obj.sel(source='asf',band='vh').mean(dim=['x','y'])).plot.scatter(x='time', ax=ax[1], label='asf',
                                                                                                  c='b',alpha=0.75)
s1_tools.power_to_db(comparison_obj.sel(source='pc',band='vh').mean(dim=['x','y'])).plot.scatter(x='time', ax=ax[1], label='pc',
                                                                                                 c='r',alpha=0.75)
ax[0].legend(loc='lower right', bbox_to_anchor=([1,-.25,0,0]))

for i in range(len(ax)):
    ax[i].set_xlabel(None)
    ax[i].set_ylabel('dB');

ax[0].set_title('VV')
ax[1].set_title('VH')

fig.supxlabel('Time')
#fig.supylabel('dB')
fig.suptitle('Comparing mean VV and VH backscatter over time from PC (red) and ASF (blue) datasets', fontsize=14, y=1.05);

{{conclusion}}